## Using local spark session
(just as an example that spark can be used locally without managing/connecting to cluster,also used for reading local files in folder testdata)

In [1]:
import pyspark
import pyspark.sql.functions as F
from pyspark.sql import SparkSession

as in notebook 02 - pásek a kšandy, probably not needed at all

In [2]:
!export PYSPARK_SUBMIT_ARGS='--conf spark.sql.extensions=org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions\
        --conf spark.sql.catalog.spark_catalog=org.apache.iceberg.spark.SparkSessionCatalog\
        --conf spark.sql.catalog.spark_catalog.type=hive\
        --conf spark.sql.catalog.iceberg_catalog.type=jdbc\
        --conf spark.sql.catalog.iceberg_catalog.uri=jdbc:postgresql://postgres:5432/iceberg_catalog\
        --conf spark.sql.catalog.iceberg_catalog.jdbc.user=iceberg\
        --conf spark.sql.catalog.iceberg_catalog.jdbc.password=icebergpassword\
        --conf spark.sql.catalog.iceberg_catalog.driver=org.postgresql.Driver\
        --conf spark.sql.catalog.iceberg_catalog.warehouse=s3a://iceberg-warehouse/\
        --conf spark.hadoop.fs.s3a.endpoint=http://minio:9000\
        --conf spark.hadoop.fs.s3a.access.key=admin\
        --conf spark.hadoop.fs.s3a.secret.key=password\
        --conf spark.hadoop.fs.s3a.path.style.access=true\
        --conf spark.hadoop.fs.s3a.connection.ssl.enabled=false\
        --conf spark.driver.extraJavaOptions="-Daws.region=eu-central-1 -Daws.overrideDefaultRegion=true"\
        --conf spark.executor.extraJavaOptions="-Daws.region=eu-central-1 -Daws.overrideDefaultRegion=true"\
        --packages org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.9.1,org.apache.hadoop:hadoop-aws:3.3.4,org.postgresql:postgresql:42.6.0,org.apache.spark:spark-avro_2.12:3.5.6\
        pyspark-shell'

In [3]:
import os
os.environ['PYSPARK_SUBMIT_ARGS']='''--conf spark.sql.extensions=org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions\
        --conf spark.sql.catalog.spark_catalog=org.apache.iceberg.spark.SparkSessionCatalog\
        --conf spark.sql.catalog.spark_catalog.type=hive\
        --conf spark.sql.catalog.iceberg_catalog.type=jdbc\
        --conf spark.sql.catalog.iceberg_catalog.uri=jdbc:postgresql://postgres:5432/iceberg_catalog\
        --conf spark.sql.catalog.iceberg_catalog.jdbc.user=iceberg\
        --conf spark.sql.catalog.iceberg_catalog.jdbc.password=icebergpassword\
        --conf spark.sql.catalog.iceberg_catalog.driver=org.postgresql.Driver\
        --conf spark.sql.catalog.iceberg_catalog.warehouse=s3a://iceberg-warehouse/\
        --conf spark.hadoop.fs.s3a.endpoint=http://minio:9000\
        --conf spark.hadoop.fs.s3a.access.key=admin\
        --conf spark.hadoop.fs.s3a.secret.key=password\
        --conf spark.hadoop.fs.s3a.path.style.access=true\
        --conf spark.hadoop.fs.s3a.connection.ssl.enabled=false\
        --conf spark.driver.extraJavaOptions="-Daws.region=eu-central-1 -Daws.overrideDefaultRegion=true"\
        --conf spark.executor.extraJavaOptions="-Daws.region=eu-central-1 -Daws.overrideDefaultRegion=true"\
        --packages org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.9.1,org.apache.hadoop:hadoop-aws:3.3.4,org.postgresql:postgresql:42.6.0,org.apache.spark:spark-avro_2.12:3.5.6\
        pyspark-shell'''

#### create spark session

local session is defined by `.master("local[number_of_cores]")`, to use all available cores, write `"local[*]"` 

In [4]:
spark = (SparkSession
         .builder
         .master("local[4]")
         .config("spark.executor.memory", "6g")
         .config("spark.driver.memory", "6g")
         .config("spark.hadoop.fs.s3a.endpoint.region","eu-central-1")
         .appName("spark-local")
        ).getOrCreate()

:: loading settings :: url = jar:file:/opt/conda/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
org.postgresql#postgresql added as a dependency
org.apache.spark#spark-avro_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-565c8e27-8b53-4599-a628-3e34356241f6;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.9.1 in central
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
	found org.postgresql#postgresql;42.6.0 in central
	found org.checkerframework#checker-qual;3.31.0 in central
	found org.apache.spark#spark-avro_2.12;3.5.6 in central
	found org.tukaani#xz;1.9 in central
:: resolution report :: resolve 118ms :: artifacts dl 7ms
	:: modules in use:
	

again, change the url to localhost:14040-14043, if the session from notebook 02 is still running, it will most probably be [localhost:14041](http://localhost:14041) 

In [5]:
spark

let's create a table from the json structure
- first explode the array 'records'
- then transform the struct field with select "col.*"
- then transform string fields to dates

In [6]:
# read the folder with jsons
df = spark.read.json("testdata/")
df.printSchema()

root
 |-- dataset: string (nullable = true)
 |-- limit: long (nullable = true)
 |-- records: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- HourDK: string (nullable = true)
 |    |    |-- HourUTC: string (nullable = true)
 |    |    |-- PriceArea: string (nullable = true)
 |    |    |-- SpotPriceDKK: double (nullable = true)
 |    |    |-- SpotPriceEUR: double (nullable = true)
 |-- sort: string (nullable = true)
 |-- total: long (nullable = true)



In [7]:
# explode the array records
df2=df.select(F.explode("records"))
df2.printSchema()

root
 |-- col: struct (nullable = true)
 |    |-- HourDK: string (nullable = true)
 |    |-- HourUTC: string (nullable = true)
 |    |-- PriceArea: string (nullable = true)
 |    |-- SpotPriceDKK: double (nullable = true)
 |    |-- SpotPriceEUR: double (nullable = true)



In [8]:
# select only the columns in column `col`
df3=df2.select(F.col("col.*"))
df3.printSchema()

root
 |-- HourDK: string (nullable = true)
 |-- HourUTC: string (nullable = true)
 |-- PriceArea: string (nullable = true)
 |-- SpotPriceDKK: double (nullable = true)
 |-- SpotPriceEUR: double (nullable = true)



In [9]:
# convert strings
df4=df3.select(F.to_timestamp_ntz("HourDK").alias("HourDK"),               
            F.to_timestamp_ntz("HourUTC").alias("HourUTC"),
           "PriceArea", "SpotPriceDKK","SpotPriceEUR")
df4.printSchema()

root
 |-- HourDK: timestamp_ntz (nullable = true)
 |-- HourUTC: timestamp_ntz (nullable = true)
 |-- PriceArea: string (nullable = true)
 |-- SpotPriceDKK: double (nullable = true)
 |-- SpotPriceEUR: double (nullable = true)



In [10]:
# show few rows
df4.limit(10).toPandas()

,HourDK,HourUTC,PriceArea,SpotPriceDKK,SpotPriceEUR
0,2023-10-31 20:00:00,2023-10-31 19:00:00,SYSTEM,482.980011,64.709999
1,2023-10-31 20:00:00,2023-10-31 19:00:00,SE3,147.559998,19.770000
2,2023-10-31 20:00:00,2023-10-31 19:00:00,DK2,939.530029,125.879997
3,2023-10-31 20:00:00,2023-10-31 19:00:00,DK1,939.530029,125.879997
4,2023-10-31 20:00:00,2023-10-31 19:00:00,SE4,147.559998,19.770000
5,2023-10-31 20:00:00,2023-10-31 19:00:00,DE,939.530029,125.879997
6,2023-10-31 19:00:00,2023-10-31 18:00:00,DE,1141.349976,152.919998
7,2023-10-31 19:00:00,2023-10-31 18:00:00,SE4,165.320007,22.150000
8,2023-10-31 19:00:00,2023-10-31 18:00:00,DK1,1141.349976,152.919998
9,2023-10-31 19:00:00,2023-10-31 18:00:00,DK2,1141.349976,152.919998


In [11]:
df4.describe().show(truncate=False)

[Stage 2:===============================================>           (4 + 1) / 5]

+-------+---------+------------------+-----------------+
|summary|PriceArea|SpotPriceDKK      |SpotPriceEUR     |
+-------+---------+------------------+-----------------+
|count  |1794951  |1494238           |1494262          |
|mean   |NULL     |339.9898769339158 |45.64850028315928|
|stddev |NULL     |374.63314435438866|50.34498024112369|
|min    |DE       |-3723.469971      |-500.019989      |
|max    |SYSTEM   |18173.359375      |2436.629883      |
+-------+---------+------------------+-----------------+



In [12]:
# let's do it in one step
df = (spark.read.json("testdata/")
        .select(F.explode("records"))
        .select(F.col("col.*"))
        .select(F.to_timestamp_ntz("HourDK").alias("HourDK"),
            F.to_timestamp_ntz("HourUTC").alias("HourUTC"),
           "PriceArea", "SpotPriceDKK","SpotPriceEUR")
     )

In [13]:
df.count()

1794951

In [14]:
df.printSchema()

root
 |-- HourDK: timestamp_ntz (nullable = true)
 |-- HourUTC: timestamp_ntz (nullable = true)
 |-- PriceArea: string (nullable = true)
 |-- SpotPriceDKK: double (nullable = true)
 |-- SpotPriceEUR: double (nullable = true)



**perfect**

now select iceberg catalog to work with and create db for the dataframe

In [15]:
spark.catalog.listCatalogs()

[CatalogMetadata(name='spark_catalog', description=None)]

we can see only spark sees only the default catalog, so we now set the iceberg_catalog:

In [16]:
spark.conf.set("spark.sql.catalog.iceberg_catalog", "org.apache.iceberg.spark.SparkCatalog")
spark.conf.set("spark.sql.catalog.iceberg_catalog.uri", "jdbc:postgresql://postgres:5432/iceberg_catalog")
spark.conf.set("spark.sql.catalog.iceberg_catalog.jdbc.user", "iceberg") 
spark.conf.set("spark.sql.catalog.iceberg_catalog.jdbc.password", "icebergpassword") 
spark.conf.set("spark.sql.catalog.iceberg_catalog.warehouse", "s3a://iceberg-warehouse/")

In [17]:
spark.catalog.setCurrentCatalog('iceberg_catalog')

In [18]:
spark.catalog.listCatalogs()

[CatalogMetadata(name='iceberg_catalog', description=None),
 CatalogMetadata(name='spark_catalog', description=None)]

In [19]:
spark.catalog.listDatabases()

[Database(name='spark_schema', catalog='iceberg_catalog', description=None, locationUri='s3a://iceberg-warehouse/spark_schema'),
 Database(name='trino_schema', catalog='iceberg_catalog', description=None, locationUri='s3a://iceberg-warehouse/trino_schema/')]

ok, let's do this on bigger data

and save it to minio using iceberg

In [21]:
spark.catalog.listTables("spark_schema")

25/07/11 13:41:35 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


[Table(name='spark_orders', catalog='iceberg_catalog', namespace=['spark_schema'], description=None, tableType='MANAGED', isTemporary=False)]

In [22]:
spark.catalog.listTables("trino_schema")

[Table(name='employees', catalog='iceberg_catalog', namespace=['trino_schema'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='events', catalog='iceberg_catalog', namespace=['trino_schema'], description=None, tableType='MANAGED', isTemporary=False)]

In [23]:
(df.writeTo(f"iceberg_catalog.spark_schema.elspotprices")
    .using("iceberg")
    .partitionedBy(F.col("PriceArea"),F.months(F.col("HourUTC")))
).createOrReplace()

25/07/11 13:42:11 WARN S3ABlockOutputStream: Application invoked the Syncable API against stream writing to spark_schema/elspotprices/data/PriceArea=DK2/HourUTC_month=2021-11/00002-43-c7fc0f2a-61b0-4a8a-9ead-1f0a47f8311c-0-00007.parquet. This is unsupported
                                                                                

25/06/16 11:26:15 WARN Tasks: Retrying task after failure: sleepTimeMs=5437 Failed to open input stream for file: s3a://iceberg-warehouse/prices/elspotprices1000000/metadata/00000-b521a475-bd9e-40c1-bc3d-0930f611a5ed.metadata.json
org.apache.iceberg.exceptions.RuntimeIOException: Failed to open input stream for file: s3a://iceberg-warehouse/prices/elspotprices1000000/metadata/00000-b521a475-bd9e-40c1-bc3d-0930f611a5ed.metadata.json
	at org.apache.iceberg.hadoop.HadoopInputFile.newStream(HadoopInputFile.java:187)
	at org.apache.iceberg.TableMetadataParser.read(TableMetadataParser.java:290)
	at org.apache.iceberg.TableMetadataParser.read(TableMetadataParser.java:284)
	at org.apache.iceberg.BaseMetastoreTableOperations.lambda$refreshFromMetadataLocation$0(BaseMetastoreTableOperations.java:180)
	at org.apache.iceberg.BaseMetastoreTableOperations.lambda$refreshFromMetadataLocation$1(BaseMetastoreTableOperations.java:199)
	at org.apache.iceberg.util.Tasks$Builder.runTaskWithRetry(Tasks.java:

25/06/16 11:26:20 WARN Tasks: Retrying task after failure: sleepTimeMs=5471 Failed to open input stream for file: s3a://iceberg-warehouse/prices/elspotprices1000000/metadata/00000-b521a475-bd9e-40c1-bc3d-0930f611a5ed.metadata.json
org.apache.iceberg.exceptions.RuntimeIOException: Failed to open input stream for file: s3a://iceberg-warehouse/prices/elspotprices1000000/metadata/00000-b521a475-bd9e-40c1-bc3d-0930f611a5ed.metadata.json
	at org.apache.iceberg.hadoop.HadoopInputFile.newStream(HadoopInputFile.java:187)
	at org.apache.iceberg.TableMetadataParser.read(TableMetadataParser.java:290)
	at org.apache.iceberg.TableMetadataParser.read(TableMetadataParser.java:284)
	at org.apache.iceberg.BaseMetastoreTableOperations.lambda$refreshFromMetadataLocation$0(BaseMetastoreTableOperations.java:180)
	at org.apache.iceberg.BaseMetastoreTableOperations.lambda$refreshFromMetadataLocation$1(BaseMetastoreTableOperations.java:199)
	at org.apache.iceberg.util.Tasks$Builder.runTaskWithRetry(Tasks.java:

25/06/16 11:26:26 WARN Tasks: Retrying task after failure: sleepTimeMs=5272 Failed to open input stream for file: s3a://iceberg-warehouse/prices/elspotprices1000000/metadata/00000-b521a475-bd9e-40c1-bc3d-0930f611a5ed.metadata.json
org.apache.iceberg.exceptions.RuntimeIOException: Failed to open input stream for file: s3a://iceberg-warehouse/prices/elspotprices1000000/metadata/00000-b521a475-bd9e-40c1-bc3d-0930f611a5ed.metadata.json
	at org.apache.iceberg.hadoop.HadoopInputFile.newStream(HadoopInputFile.java:187)
	at org.apache.iceberg.TableMetadataParser.read(TableMetadataParser.java:290)
	at org.apache.iceberg.TableMetadataParser.read(TableMetadataParser.java:284)
	at org.apache.iceberg.BaseMetastoreTableOperations.lambda$refreshFromMetadataLocation$0(BaseMetastoreTableOperations.java:180)
	at org.apache.iceberg.BaseMetastoreTableOperations.lambda$refreshFromMetadataLocation$1(BaseMetastoreTableOperations.java:199)
	at org.apache.iceberg.util.Tasks$Builder.runTaskWithRetry(Tasks.java:

25/06/16 11:26:31 WARN Tasks: Retrying task after failure: sleepTimeMs=5115 Failed to open input stream for file: s3a://iceberg-warehouse/prices/elspotprices1000000/metadata/00000-b521a475-bd9e-40c1-bc3d-0930f611a5ed.metadata.json
org.apache.iceberg.exceptions.RuntimeIOException: Failed to open input stream for file: s3a://iceberg-warehouse/prices/elspotprices1000000/metadata/00000-b521a475-bd9e-40c1-bc3d-0930f611a5ed.metadata.json
	at org.apache.iceberg.hadoop.HadoopInputFile.newStream(HadoopInputFile.java:187)
	at org.apache.iceberg.TableMetadataParser.read(TableMetadataParser.java:290)
	at org.apache.iceberg.TableMetadataParser.read(TableMetadataParser.java:284)
	at org.apache.iceberg.BaseMetastoreTableOperations.lambda$refreshFromMetadataLocation$0(BaseMetastoreTableOperations.java:180)
	at org.apache.iceberg.BaseMetastoreTableOperations.lambda$refreshFromMetadataLocation$1(BaseMetastoreTableOperations.java:199)
	at org.apache.iceberg.util.Tasks$Builder.runTaskWithRetry(Tasks.java:

...

25/06/16 11:26:52 WARN Tasks: Retrying task after failure: sleepTimeMs=5049 Failed to open input stream for file: s3a://iceberg-warehouse/prices/elspotprices1000000/metadata/00000-b521a475-bd9e-40c1-bc3d-0930f611a5ed.metadata.json
org.apache.iceberg.exceptions.RuntimeIOException: Failed to open input stream for file: s3a://iceberg-warehouse/prices/elspotprices1000000/metadata/00000-b521a475-bd9e-40c1-bc3d-0930f611a5ed.metadata.json
	at org.apache.iceberg.hadoop.HadoopInputFile.newStream(HadoopInputFile.java:187)
	at org.apache.iceberg.TableMetadataParser.read(TableMetadataParser.java:290)
	at org.apache.iceberg.TableMetadataParser.read(TableMetadataParser.java:284)
	at org.apache.iceberg.BaseMetastoreTableOperations.lambda$refreshFromMetadataLocation$0(BaseMetastoreTableOperations.java:180)
	at org.apache.iceberg.BaseMetastoreTableOperations.lambda$refreshFromMetadataLocation$1(BaseMetastoreTableOperations.java:199)
	at org.apache.iceberg.util.Tasks$Builder.runTaskWithRetry(Tasks.java:

25/06/16 11:26:57 WARN Tasks: Retrying task after failure: sleepTimeMs=5351 Failed to open input stream for file: s3a://iceberg-warehouse/prices/elspotprices1000000/metadata/00000-b521a475-bd9e-40c1-bc3d-0930f611a5ed.metadata.json
org.apache.iceberg.exceptions.RuntimeIOException: Failed to open input stream for file: s3a://iceberg-warehouse/prices/elspotprices1000000/metadata/00000-b521a475-bd9e-40c1-bc3d-0930f611a5ed.metadata.json
	at org.apache.iceberg.hadoop.HadoopInputFile.newStream(HadoopInputFile.java:187)
	at org.apache.iceberg.TableMetadataParser.read(TableMetadataParser.java:290)
	at org.apache.iceberg.TableMetadataParser.read(TableMetadataParser.java:284)
	at org.apache.iceberg.BaseMetastoreTableOperations.lambda$refreshFromMetadataLocation$0(BaseMetastoreTableOperations.java:180)
	at org.apache.iceberg.BaseMetastoreTableOperations.lambda$refreshFromMetadataLocation$1(BaseMetastoreTableOperations.java:199)
	at org.apache.iceberg.util.Tasks$Builder.runTaskWithRetry(Tasks.java:

25/06/16 11:27:02 WARN Tasks: Retrying task after failure: sleepTimeMs=5254 Failed to open input stream for file: s3a://iceberg-warehouse/prices/elspotprices1000000/metadata/00000-b521a475-bd9e-40c1-bc3d-0930f611a5ed.metadata.json
org.apache.iceberg.exceptions.RuntimeIOException: Failed to open input stream for file: s3a://iceberg-warehouse/prices/elspotprices1000000/metadata/00000-b521a475-bd9e-40c1-bc3d-0930f611a5ed.metadata.json
	at org.apache.iceberg.hadoop.HadoopInputFile.newStream(HadoopInputFile.java:187)
	at org.apache.iceberg.TableMetadataParser.read(TableMetadataParser.java:290)
	at org.apache.iceberg.TableMetadataParser.read(TableMetadataParser.java:284)
	at org.apache.iceberg.BaseMetastoreTableOperations.lambda$refreshFromMetadataLocation$0(BaseMetastoreTableOperations.java:180)
	at org.apache.iceberg.BaseMetastoreTableOperations.lambda$refreshFromMetadataLocation$1(BaseMetastoreTableOperations.java:199)
	at org.apache.iceberg.util.Tasks$Builder.runTaskWithRetry(Tasks.java:

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/conda/lib/python3.12/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.12/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.12/socket.py", line 720, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

- not sure what the warning means, but the table is created, so whatever

**different way to create a table from dataframe - first register the dataframe as view, then use CTAS to write it to iceberg with limit 0 to create its structure, then writeTo...append():**

In [24]:
df.createTempView("elspotprices2")

In [25]:
spark.sql("create table iceberg_catalog.spark_schema.elspotprices2 as select * from elspotprices2 limit 0")


DataFrame[]

showcasing compaction on bigger data, deliberately saving the dataframe into small files, with 1000 records each

In [26]:
df.repartition(1000).writeTo(f"iceberg_catalog.spark_schema.elspotprices2").option("maxRecordsPerFile", 1000).using('iceberg').append()

In [27]:
spark.sql(f"""
    SELECT file_path, record_count, file_size_in_bytes
    FROM iceberg_catalog.spark_schema.elspotprices2.files
""").show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------+------------+------------------+
|file_path                                                                                                            |record_count|file_size_in_bytes|
+---------------------------------------------------------------------------------------------------------------------+------------+------------------+
|s3a://iceberg-warehouse/spark_schema/elspotprices2/data/00000-51-a7b6b968-aefc-4abf-b55c-715eb4934caa-0-00001.parquet|1795        |37486             |
|s3a://iceberg-warehouse/spark_schema/elspotprices2/data/00001-52-a7b6b968-aefc-4abf-b55c-715eb4934caa-0-00001.parquet|1795        |37296             |
|s3a://iceberg-warehouse/spark_schema/elspotprices2/data/00002-53-a7b6b968-aefc-4abf-b55c-715eb4934caa-0-00001.parquet|1795        |37080             |
|s3a://iceberg-warehouse/spark_schema/elspotprices2/data/00003-54-a7b6b968-aefc-4abf-b55

In [31]:
result_df = spark.sql(f"""CALL iceberg_catalog.system.rewrite_data_files(
                      table => 'spark_schema.elspotprices2', 
                      strategy => 'sort', sort_order => 'HourUTC ASC',
                      options => map('min-input-files','5'))""")

+--------------------------+----------------------+---------------------+-----------------------+
|rewritten_data_files_count|added_data_files_count|rewritten_bytes_count|failed_data_files_count|
+--------------------------+----------------------+---------------------+-----------------------+
|0                         |0                     |0                    |0                      |
+--------------------------+----------------------+---------------------+-----------------------+



In [33]:
# summary of what happened:
result_df.show(truncate=False)

+--------------------------+----------------------+---------------------+-----------------------+
|rewritten_data_files_count|added_data_files_count|rewritten_bytes_count|failed_data_files_count|
+--------------------------+----------------------+---------------------+-----------------------+
|0                         |0                     |0                    |0                      |
+--------------------------+----------------------+---------------------+-----------------------+



In [53]:
spark.sql(f"""SELECT file_path, record_count, file_size_in_bytes
FROM iceberg_catalog.spark_schema.elspotprices2.files""").show(truncate=False)


+-----------------------------------------------------------------------------------------------------------------------+------------+------------------+
|file_path                                                                                                              |record_count|file_size_in_bytes|
+-----------------------------------------------------------------------------------------------------------------------+------------+------------------+
|s3a://iceberg-warehouse/spark_schema/elspotprices2/data/00000-1084-41377737-1c8d-4269-a25b-29de199fdba6-0-00001.parquet|1794951     |11741509          |
+-----------------------------------------------------------------------------------------------------------------------+------------+------------------+



view snapshots

In [75]:
spark.sql("SELECT * FROM iceberg_catalog.spark_schema.elspotprices2.snapshots ORDER BY committed_at DESC").show(truncate=True)
spark.sql("SELECT snapshot_id, manifest_list FROM iceberg_catalog.spark_schema.elspotprices2.snapshots ORDER BY committed_at DESC").show(truncate=False)
spark.sql("SELECT snapshot_id, summary FROM iceberg_catalog.spark_schema.elspotprices2.snapshots ORDER BY committed_at DESC").toPandas()


+--------------------+-------------------+-------------------+---------+--------------------+--------------------+
|        committed_at|        snapshot_id|          parent_id|operation|       manifest_list|             summary|
+--------------------+-------------------+-------------------+---------+--------------------+--------------------+
|2025-07-11 13:51:...|6092011551680698518|4259939819494638523|  replace|s3a://iceberg-war...|{added-data-files...|
|2025-07-11 13:49:...|4259939819494638523| 673444911004302166|   append|s3a://iceberg-war...|{spark.app.id -> ...|
|2025-07-11 13:43:...| 673444911004302166|               NULL|   append|s3a://iceberg-war...|{spark.app.id -> ...|
+--------------------+-------------------+-------------------+---------+--------------------+--------------------+

+-------------------+--------------------------------------------------------------------------------------------------------------------------------+
|snapshot_id        |manifest_list         

,snapshot_id,summary
0,6092011551680698518,"{'engine-version': '3.5.6', 'added-data-files': '1', 'total-equality-deletes': '0', 'app-id': 'local-1752241220564', 'added-records': '1794951', 'deleted-data-files': '1000', 'deleted-records': '1794951', 'total-records': '1794951', 'removed-files-size': '37125750', 'changed-partition-count': '1..."
1,4259939819494638523,"{'engine-version': '3.5.6', 'added-data-files': '1000', 'total-equality-deletes': '0', 'app-id': 'local-1752241220564', 'added-records': '1794951', 'total-records': '1794951', 'spark.app.id': 'local-1752241220564', 'changed-partition-count': '1', 'engine-name': 'spark', 'total-position-deletes':..."
2,673444911004302166,"{'spark.app.id': 'local-1752241220564', 'changed-partition-count': '0', 'engine-version': '3.5.6', 'total-equality-deletes': '0', 'app-id': 'local-1752241220564', 'engine-name': 'spark', 'total-position-deletes': '0', 'total-delete-files': '0', 'iceberg-version': 'Apache Iceberg 1.9.1 (commit f4..."


In [76]:
snapshots_df = spark.sql("SELECT * FROM iceberg_catalog.spark_schema.elspotprices2.snapshots ORDER BY committed_at DESC")
snapshots_df.printSchema()

root
 |-- committed_at: timestamp (nullable = false)
 |-- snapshot_id: long (nullable = false)
 |-- parent_id: long (nullable = true)
 |-- operation: string (nullable = true)
 |-- manifest_list: string (nullable = true)
 |-- summary: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = false)



to display summary in human friendly way let's explode it and pivot key:

In [77]:
snpshts_ex_summ_df = (snapshots_df
                      .select(*[F.col(f"{col}") for col in snapshots_df.columns if col!="summary"], F.explode("summary"))
                      .groupBy(*[F.col(f"{col}") for col in snapshots_df.columns if col!="summary"])
                          .pivot("key")
                          .agg(F.first("value")))

In [78]:
import pandas
pandas.set_option("display.max_colwidth",300)
snpshts_ex_summ_df.toPandas()

,committed_at,snapshot_id,parent_id,operation,manifest_list,added-data-files,added-files-size,added-records,app-id,changed-partition-count,deleted-data-files,deleted-records,engine-name,engine-version,iceberg-version,removed-files-size,spark.app.id,total-data-files,total-delete-files,total-equality-deletes,total-files-size,total-position-deletes,total-records
0,2025-07-11 13:43:17.153,673444911004302166,NaN,append,s3a://iceberg-warehouse/spark_schema/elspotprices2/metadata/snap-673444911004302166-1-496b239d-720b-4aa9-a126-deddff4403b4.avro,None,None,None,local-1752241220564,0,None,None,spark,3.5.6,Apache Iceberg 1.9.1 (commit f40208ae6fb2f33e578c2637d3dea1db18739f31),None,local-1752241220564,0,0,0,0,0,0
1,2025-07-11 13:49:36.775,4259939819494638523,6.734449e+17,append,s3a://iceberg-warehouse/spark_schema/elspotprices2/metadata/snap-4259939819494638523-1-81d488d9-0cdd-4e86-bf7e-2267e6c0029a.avro,1000,37125750,1794951,local-1752241220564,1,None,None,spark,3.5.6,Apache Iceberg 1.9.1 (commit f40208ae6fb2f33e578c2637d3dea1db18739f31),None,local-1752241220564,1000,0,0,37125750,0,1794951
2,2025-07-11 13:51:42.775,6092011551680698518,4.259940e+18,replace,s3a://iceberg-warehouse/spark_schema/elspotprices2/metadata/snap-6092011551680698518-1-d1c67091-97e7-4262-9fe3-4c1201a302a1.avro,1,11741509,1794951,local-1752241220564,1,1000,1794951,spark,3.5.6,Apache Iceberg 1.9.1 (commit f40208ae6fb2f33e578c2637d3dea1db18739f31),37125750,None,1,0,0,11741509,0,1794951


In [97]:
spark.catalog.setCurrentCatalog("iceberg_catalog")

In [102]:
display(spark.catalog.listTables("spark_schema"))
print()
display(spark.catalog.listTables("trino_schema"))

[Table(name='elspotprices', catalog='iceberg_catalog', namespace=['spark_schema'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='elspotprices2', catalog='iceberg_catalog', namespace=['spark_schema'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='spark_orders', catalog='iceberg_catalog', namespace=['spark_schema'], description=None, tableType='MANAGED', isTemporary=False)]

[Table(name='employees', catalog='iceberg_catalog', namespace=['trino_schema'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='events', catalog='iceberg_catalog', namespace=['trino_schema'], description=None, tableType='MANAGED', isTemporary=False)]

In [106]:
spark.catalog.setCurrentCatalog("spark_catalog")
display(spark.catalog.listDatabases())
display(spark.catalog.listTables("default"))

[Database(name='default', catalog='spark_catalog', description='default database', locationUri='file:/home/jovyan/notebooks/spark-warehouse')]

[Table(name='elspotprices2', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

oh, what we didn't try is to read spark-created table with trino, but why shouldn't that work..
anyway:

In [107]:
from sqlalchemy import create_engine, text
import pandas as pd

TRINO_HOST = 'trino-coordinator' # Service name in docker-compose
TRINO_PORT = 8080
TRINO_USER = 'testuser' # Can be any string, Trino by default doesn't enforce auth in this setup
CATALOG = 'iceberg' # Catalog name as defined in iceberg.properties

# Connection string for Trino
trino_conn_str = f'trino://{TRINO_USER}@{TRINO_HOST}:{TRINO_PORT}/{CATALOG}'
engine = create_engine(trino_conn_str)

In [108]:
def run_trino_query(query, fetch_results=True):
    """Executes a Trino query and optionally fetches results into a Pandas DataFrame."""
    with engine.connect() as connection:
        # For queries that modify data or schema, autocommit is usually the default or not needed to be set explicitly for Trino
        # For DML/DDL, we might not always fetch results
        result_proxy = connection.execute(text(query))
        if fetch_results and result_proxy.returns_rows:
            df = pd.DataFrame(result_proxy.fetchall(), columns=result_proxy.keys())
            return df
        elif fetch_results: # No rows returned but fetch_results was true
            return pd.DataFrame(columns=result_proxy.keys() if result_proxy.returns_rows else [])
        else:
            print(f"Query executed successfully (returns_rows={result_proxy.returns_rows}).")
            # For DDL/DML, we might want to check row count if available
            # print(f"Rows affected (approx): {result_proxy.rowcount}") # rowcount might not be reliable for all statements/drivers
            return None

print(f"Connected to Trino: {trino_conn_str}")

Connected to Trino: trino://testuser@trino-coordinator:8080/iceberg


In [110]:
print("\nAvailable schemas in Iceberg catalog:")
schemas_df = run_trino_query(f"SHOW SCHEMAS FROM {CATALOG}")
print(schemas_df)


Available schemas in Iceberg catalog:
               Schema
0  information_schema
1        spark_schema
2              system
3        trino_schema


In [111]:
SCHEMA_NAME = 'spark_schema'

In [112]:
run_trino_query(f"show tables from {CATALOG}.{SCHEMA_NAME}")

,Table
0,elspotprices
1,elspotprices2
2,spark_orders


In [113]:
trino_df=run_trino_query(f"select * from {CATALOG}.{SCHEMA_NAME}.elspotprices")

In [114]:
trino_df

,hourdk,hourutc,pricearea,spotpricedkk,spotpriceeur
0,2025-07-01 01:00:00,2025-06-30 23:00:00,DK1,710.930110,95.290001
1,2025-04-01 01:00:00,2025-03-31 23:00:00,DK1,709.001210,95.019997
2,2022-04-01 01:00:00,2022-03-31 23:00:00,DK2,741.070007,99.639999
3,2025-04-01 00:00:00,2025-03-31 22:00:00,DK1,757.800081,101.559998
4,2025-03-31 23:00:00,2025-03-31 21:00:00,DK1,763.589412,102.339996
...,...,...,...,...,...
1794946,1999-12-01 05:00:00,1999-12-01 04:00:00,DE,NaN,NaN
1794947,1999-12-01 04:00:00,1999-12-01 03:00:00,DE,NaN,NaN
1794948,1999-12-01 03:00:00,1999-12-01 02:00:00,DE,NaN,NaN
1794949,1999-12-01 02:00:00,1999-12-01 01:00:00,DE,NaN,NaN


ok, this is the end